### Домашнее задание <a class="anchor" id="hw"></a><center>

In [60]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

__Задача:__ подойдет ли репетитор для подготовки к экзамену по математике

In [61]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [62]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [63]:
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

1 метод

In [64]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [92]:
eps = 1e-8

np.clip([1, 0, 0], eps, 1 - eps)

array([9.9999999e-01, 1.0000000e-08, 1.0000000e-08])

In [65]:
def calc_logloss(y, y_pred):
    eps = 1e-8
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

2 метод

In [66]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

In [67]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [68]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [69]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [70]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [71]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

1 метод

аналог GridSearch

In [72]:
from tqdm import tqdm

In [77]:
iterations = np.logspace(2, 5, 4, dtype=np.int)
etas = np.linspace(1e-2, 5, 10)

best_error = np.inf
best_params = {}

for iteration in tqdm(iterations):
    for eta in etas:
        W, error = eval_model(X_st, y, iterations=iteration, eta=eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': iteration,
                'eta': eta
            }

print(f'Ошибка {best_error} при параметрах {best_params}')

100%|██████████| 4/4 [00:54<00:00, 13.66s/it]

Ошибка 0.003985684540654134 при параметрах {'iteration': 100000, 'eta': 5.0}


2 метод

Вручную

In [101]:
W, error = eval_model(X_st, y, iterations=100, eta=4)

0 [ 0.35471749 -1.22154236 -0.28601831  1.02511075] 0.760958797591889
10 [ 1.74524510e-03 -2.28338321e+00 -6.79637042e-01  3.22729351e+00] 0.3322369010760909
20 [ 0.10463396 -2.65190818 -1.1068567   4.25259661] 0.29290854740108785
30 [ 0.21466455 -2.92848549 -1.47776456  4.9846682 ] 0.2731021569812503
40 [ 0.32011366 -3.15150546 -1.78839465  5.59488481] 0.25952996520454946
50 [ 0.41898795 -3.34761692 -2.05973011  6.13321495] 0.2490584530186956
60 [ 0.51162553 -3.5276709  -2.30361599  6.62353031] 0.24042303880559399
70 [ 0.59877762 -3.69672027 -2.52725542  7.07891825] 0.23300566324010222
80 [ 0.68116596 -3.85740865 -2.7352801   7.50725774] 0.2264637706000625
90 [ 0.7593943  -4.01127147 -2.9307987   7.91363943] 0.22058939385110884


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [74]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [75]:
calc_pred_proba(W, X_st)

array([0.33270288, 0.17871258, 0.99197788, 0.00306022, 0.67845939,
       0.31632068, 0.997323  , 0.11952076, 0.38019789, 0.9777108 ])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [99]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [102]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

**5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

*Accuracy*

In [103]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [104]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0 0 1 0 1 0 1 0 1 1]
Predicted [0 0 1 0 1 0 1 0 0 1]


In [105]:
accuracy(y, pred)

0.9

*Confusion Matrix*

In [81]:
# TP  FP
# FN  TN

In [82]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1
    return cm

In [83]:
cm = confusion_matrix(y, pred)
cm

array([[4., 0.],
       [1., 5.]])

*Precision*

In [84]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [85]:
precision(y, pred)

1.0

*Recall*

In [86]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [87]:
recall(y, pred)

0.8

*F-score*

In [88]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [89]:
f_score(y, pred)

0.888888888888889

In [90]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


print(f'accuracy {accuracy_score(y, pred)}')
print(f'precision {precision_score(y, pred)}')
print(f'recall {recall_score(y, pred)}')
print(f'f-score {f1_score(y, pred)}')
print(f'\n {confusion_matrix(y, pred)}')

accuracy 0.9
precision 1.0
recall 0.8
f-score 0.888888888888889

 [[5 0]
 [1 4]]


**6. Могла ли модель переобучиться? Почему?**

Могла, если взять большое количество итераций, когда ошибка будет стремиться к нулю.